In [2]:
#pip install pymc3

In [2]:
import numpy as np
import torch 
import scipy.stats
import math
from scipy.stats import multivariate_normal
from torch import distributions
import pymc3 as pm
import random
import matplotlib.pyplot as plt

$\ Z_{it}|Z_{i,t-1}$ ~ $\ N$($\Omega Z_{i,t-1}$, $\ I$\)\
$\ X_{it}|Z_{i,t}$ ~ $Categorical$($\Psi Z_{it}$)

Theoretical derivation gives:\
$\ Z_{it}|X_{i,t}$ ~ $\ N$($\Omega Z_{i,t-1} + \Psi^T e_{X_{it}}^T$, $\ I$\)


In [172]:
N = 500 # num patients
T = 10 # num of hospital visits for each patients
random.seed(53)
zdim = 1
xdim = 100

Z = torch.zeros((N, T, zdim)) # latent
logPZ = torch.zeros((N, T)) 
X = torch.zeros((N, T)) # observed
PX = torch.zeros((N, T, xdim)) 
logPX = torch.zeros((N, T, xdim))
d = torch.ones(zdim,requires_grad = True) 
omega = torch.diag(d) 
Psi = torch.randn((xdim,zdim),requires_grad = True)


for i in range(N): 
    '''
    I tried to find a way to get rid of outer loop but it turns out that although 
    I could calculate mean by mean = torch.matmul(omega,Z[:,t-1]) which allows me
    to remove this loop, I still need to loop again in torch.distributions, it seems
    this function doesn't support the input argument 'mean' to be a matrix aggregating
    all the means?
    '''
    mean = torch.zeros((zdim))
    cov = torch.eye(zdim)
    Zi0 = torch.distributions.multivariate_normal.MultivariateNormal(mean,cov)
    Z[i,0] = Zit.sample()
    for t in range(1,T):
        # Zit|Zi,t-1
        mean = torch.matmul(omega,Z[i,t-1])
        cov = torch.eye(zdim)*(1e-3)
        Zit = torch.distributions.multivariate_normal.MultivariateNormal(mean,cov)
        Z[i,t] = Zit.sample()
        logPZ[i,t] = Zit.log_prob(Z[i,t])

        #Xit|Zit
        Psi_z = torch.matmul(Psi,Z[i,t])
        logPX[i,t] = Psi_z - torch.logsumexp(Psi_z,0,keepdim=True)
        PX[i,t] = torch.exp(logPX[i,t])
        Xit = torch.distributions.multivariate_normal.MultivariateNormal(PX[i,t])
        X[i,t] = Xit.sample()
        

        
        
def posterior_i(x,omega,Psi):
    n,t = x.size()
    z = torch.zeros((n,t,zdim))
    e = torch.zeros((n,t,xdim))
    logpz = torch.zeros((n, t)) 
    
    
    for i in range(n):
        mean = torch.zeros((zdim))
        cov = torch.eye(zdim)
        zi0 = torch.distributions.multivariate_normal.MultivariateNormal(mean,cov)
        z[i,0] = zit.sample()
        for j in range(1,t):
            e[i,j,int(x[i,j])] = 1 
            mean = torch.matmul(omega,z[i,j-1]) + torch.matmul(Psi.t(),e[i,j].t())
            cov = torch.eye(zdim)*(1e-3)
            zit = torch.distributions.multivariate_normal.MultivariateNormal(mean,cov)
            z[i,j] = zit.sample()
            logpz[i,j] = zit.log_prob(z[i,j])
    return z, logpz
        

    
Z_posterior = torch.zeros((N, T, zdim))
Z_posterior,log_posterior = posterior_i(X,omega,Psi)

UnboundLocalError: local variable 'zit' referenced before assignment

In [ ]:
torch.mean((Z-Z_posterior)**2)

In [ ]:
plt.scatter(Z[0,T-1,0].detach().numpy(),Z[:,T-1,0].detach().numpy())

In [165]:
Z_posterior

tensor([[[ 0.0000],
         [-1.0158],
         [-1.1527],
         ...,
         [ 0.0353],
         [-0.2036],
         [ 2.1340]],

        [[ 0.0000],
         [ 0.5678],
         [ 0.2047],
         ...,
         [-2.2459],
         [-1.7722],
         [-3.2543]],

        [[ 0.0000],
         [ 0.2982],
         [ 0.1043],
         ...,
         [-1.0452],
         [ 0.0255],
         [-0.5496]],

        ...,

        [[ 0.0000],
         [ 0.3345],
         [ 1.1453],
         ...,
         [-2.4516],
         [-1.4917],
         [-1.2832]],

        [[ 0.0000],
         [ 1.5428],
         [ 1.0803],
         ...,
         [ 2.5984],
         [ 5.5063],
         [ 8.3393]],

        [[ 0.0000],
         [ 0.2435],
         [ 2.2407],
         ...,
         [ 2.6283],
         [ 3.2842],
         [ 3.7031]]])

In [132]:
Z_posterior

tensor([[[ 0.0000],
         [ 0.4734]],

        [[ 0.0000],
         [ 0.4552]],

        [[ 0.0000],
         [-0.2342]],

        [[ 0.0000],
         [-0.2104]]])